In [2]:
import numpy as np
import pandas as pd

In [41]:
Train_data = pd.read_csv('Data_Part1/Train_data_part1.csv').values
Test_data = pd.read_csv('Data_Part1/Test_data_part1.csv').values

In [42]:
print(Train_data)

[['low' 'low' 2 'no' 'yes']
 ['low' 'med' 4 'yes' 'no']
 ['low' 'high' 4 'no' 'no']
 ['med' 'med' 4 'no' 'no']
 ['med' 'med' 4 'yes' 'yes']
 ['med' 'high' 2 'yes' 'no']
 ['high' 'med' 4 'yes' 'yes']
 ['high' 'high' 2 'yes' 'no']
 ['high' 'high' 5 'yes' 'yes']]


In [43]:
X_train = Train_data[:,:4]
Y_train = Train_data[:,4]

X_test = Test_data[:,:4]
Y_test = Test_data[:,4]

In [44]:
print(X_train)
print(Y_train)

[['low' 'low' 2 'no']
 ['low' 'med' 4 'yes']
 ['low' 'high' 4 'no']
 ['med' 'med' 4 'no']
 ['med' 'med' 4 'yes']
 ['med' 'high' 2 'yes']
 ['high' 'med' 4 'yes']
 ['high' 'high' 2 'yes']
 ['high' 'high' 5 'yes']]
['yes' 'no' 'no' 'no' 'yes' 'no' 'yes' 'no' 'yes']


In [45]:
# Conversion of strings into integer types

# Price and Maintenance are ordinal variables, Airbag is a categorical variable.
# Profitable is also a categorical variable.
#from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder

LE1 = LabelEncoder()
LE2 = LabelEncoder()
LE3 = LabelEncoder()
LE4 = LabelEncoder()


LE1.fit(X_train[:,3])
LE2.fit(Y_train)
LE3.fit(X_train[:,0])
LE4.fit(X_train[:,1])

X_train[:,3] = LE1.transform(X_train[:,3])
X_test[:,3] = LE1.transform(X_test[:,3])
Y_train = LE2.transform(Y_train)
Y_test = LE2.transform(Y_test)

X_train[:,0] = LE3.transform(X_train[:,0])
X_train[:,0] = np.where(X_train[:,0]==0,int(3),X_train[:,0])

X_test[:,0] = LE3.transform(X_test[:,0])
X_test[:,0] = np.where(X_test[:,0]==0,int(3),X_test[:,0])

X_train[:,1] = LE4.transform(X_train[:,1])
X_train[:,1] = np.where(X_train[:,1]==0,int(3),X_train[:,1])

X_test[:,1] = LE4.transform(X_test[:,1])
X_test[:,1] = np.where(X_test[:,1]==0,int(3),X_test[:,1])

In [61]:
print(X_train)
#LE3.transform(['high','low','low','med'])
print(Y_train)

[[1 1 2 0]
 [1 2 4 1]
 [1 3 4 0]
 [2 2 4 0]
 [2 2 4 1]
 [2 3 2 1]
 [3 2 4 1]
 [3 3 2 1]
 [3 3 5 1]]
[1 0 0 0 1 0 1 0 1]


In [48]:
num_features = X_train.shape[1]
num_samples = X_train.shape[0]

In [63]:
print(num_features)
print(num_samples)
features = ['price','maintenance','capacity','airbag']

4
9


In [50]:
# Make feature_values array
feature_values = []
for i in range(0,num_features):
    feature_values.append(list(set(X_train[:,i])))

In [53]:
print(np.mean(feature_values[0]))

2.0


In [194]:
def GINI(Y):
    #print(Y)
    nC1 = 0
    nC2 = 0
    if(np.shape(Y)==(np.shape(Y)[0],2)):
        for i in Y:
            if(i[1]==0):
                nC1+=1
            elif(i[1]==1):
                nC2+=1
        n = nC1 + nC2
        gini = 1 - np.power((nC1/n),2) - np.power((nC2/n),2)
        return [gini,nC1,nC2]
    else:
        for i in Y:
            if(i==0):
                nC1+=1
            elif(i==1):
                nC2+=1
        n = nC1 + nC2
        gini = 1 - np.power((nC1/n),2) - np.power((nC2/n),2)
        return [gini,nC1,nC2]

In [244]:
def split(X, Y, num_features=num_features, feature_values=feature_values, features=features):
    num_samples = X.shape[0]
    Node = []
    best_feature_idx = 0
    for i in range(0,num_features):
        L = []
        R = []
        min_gini = 1
        for j in range(0,num_samples):
            if(X[j][i]<np.mean(feature_values[i])):
                L.append([X[j],Y[j]])
            else:
                R.append([X[j],Y[j]])
        
        n1 = len(L)
        n2 = len(R)
        n = n1 + n2
        Gini_split = (n1/n)*(GINI(L)[0]) + (n2/n)*(GINI(R)[0])
        #print("\n")
        if(Gini_split<min_gini):
            min_gini = Gini_split
            best_feature_idx = i
    
    # Now we know best feature for splitting
    for j in range(0,num_samples):
        if(X[j][best_feature_idx]<np.mean(feature_values[best_feature_idx])):
            L.append([X[j],Y[j]])
        else:
            R.append([X[j],Y[j]])
    gini_L, nLC1, nLC2 = GINI(L)
    gini_R, nRC1, nRC2 = GINI(R)
    gini , nC1, nC2 = GINI(Y)
    node_class = -1
    majority_class = -1
    if(nC1>nC2):
        majority_class = 0
    else:
        majority_class = 1
    if(nC1==0):
        node_class = 1
    elif(nC2==0):
        node_class = 0
    Node_info = [gini,features[best_feature_idx],np.mean(feature_values[best_feature_idx]), node_class,majority_class]
    Node.append([Node_info,L,R])
    
    return Node

In [245]:
#print(Y_train)
Root_Node = split(X_train,Y_train)

In [246]:
# print(X_train)
# print(Y_train)

In [247]:
print(Root_Node)
print("\n",Root_Node[0][1])
print("\n",Root_Node[0][2])
print("\n",Root_Node[0][0])

[[[0.49382716049382713, 'airbag', 0.5, -1, 0], [[array([1, 1, 2, 0], dtype=object), 1], [array([1, 3, 4, 0], dtype=object), 0], [array([2, 2, 4, 0], dtype=object), 0], [array([1, 1, 2, 0], dtype=object), 1], [array([1, 3, 4, 0], dtype=object), 0], [array([2, 2, 4, 0], dtype=object), 0]], [[array([1, 2, 4, 1], dtype=object), 0], [array([2, 2, 4, 1], dtype=object), 1], [array([2, 3, 2, 1], dtype=object), 0], [array([3, 2, 4, 1], dtype=object), 1], [array([3, 3, 2, 1], dtype=object), 0], [array([3, 3, 5, 1], dtype=object), 1], [array([1, 2, 4, 1], dtype=object), 0], [array([2, 2, 4, 1], dtype=object), 1], [array([2, 3, 2, 1], dtype=object), 0], [array([3, 2, 4, 1], dtype=object), 1], [array([3, 3, 2, 1], dtype=object), 0], [array([3, 3, 5, 1], dtype=object), 1]]]]

 [[array([1, 1, 2, 0], dtype=object), 1], [array([1, 3, 4, 0], dtype=object), 0], [array([2, 2, 4, 0], dtype=object), 0], [array([1, 1, 2, 0], dtype=object), 1], [array([1, 3, 4, 0], dtype=object), 0], [array([2, 2, 4, 0], dtyp

In [202]:
# L = []
# R = []
# for j in range(0,num_samples):
#             if(X_train[j][i]<np.mean(feature_values[i])):
#                 L.append([X_train[j],Y_train[j]])
#             else:
#                 R.append([X_train[j],Y_train[j]])

In [206]:
# print(L)
# print(np.shape(L)[1])

In [205]:
# print(R)

In [249]:
print(Root_Node[0][0][1])

airbag


In [ ]:
# Node_info = Node[0][0]
# L = Node[0][1]
# R = Node[0][2]
max_depth=20
DT = []
Nodes = []
Nodes.append([Root_Node[0][0][1]])
for i in range(0,max_depth):
#     Node_info = Root_Node[0][0]
#     L_Node = Root_Node[0][1]
#     R_Node = Root_Node[0][2]
    
    L_Node = split(L)
    R_Node = split(R)
    
    Nodes.append([L_Node[0][0][1]])
    Nodes.append([R_Node[0][0][1]])